### Imports

In [132]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical
from tools import *

In [133]:
!pip install -r requirements.txt

     |████████████████████████████████| 175.3 MB 9.9 MB/s eta 0:00:013    |██                              | 10.9 MB 14.6 MB/s eta 0:00:12     |█████████████████████████       | 136.8 MB 11.6 MB/s eta 0:00:04     |█████████████████████████▏      | 137.9 MB 11.6 MB/s eta 0:00:04
     |████████████████████████████████| 15.1 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 49.1 MB 3.8 MB/s eta 0:00:011     |██████████████████████████████▌ | 46.7 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 545 kB/s eta 0:00:01    |███████▏                        | 2.8 MB 541 kB/s eta 0:00:18
  Using cached h5py-2.10.0-cp37-cp37m-macosx_10_6_intel.whl (3.0 MB)
     |████████████████████████████████| 3.0 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 777 kB 24.0 MB/s eta 0:00:01


  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=4aabc011fa239f9e59837a7b46ed2e76e5f86861ccdff12e975800a0b6d4e692
  Stored in directory: /Users/sofiapaganin/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.1
    Uninstalling numpy-1.18.1:
      Successfully uninstalled numpy-1.18.1
  Attempting uninstall: h5py
    Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1


    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.2.0.32
    Uninstalling opencv-python-4.2.0.32:
      Successfully uninstalled opencv-python-4.2.0.32
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1


In [134]:
BATCH_SIZE = 256
BUFFER_SIZE = 512
EPOCHS = 10
OUTPUT_DIR = "img"


### Loading Data

In [135]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

labels = get_label_dict()
words = get_word_labels()

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data

Loading labels

Done

Loading words

Done



### Models

In [136]:
class Generator(keras.Model):
    
    def __init__(self, random_noise_size = 100):
        super().__init__(name='generator')
        #layers
        self.input_layer = keras.layers.Dense(units = random_noise_size)
        self.dense_1 = keras.layers.Dense(units = 128)
        self.leaky_1 =  keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_2 = keras.layers.Dense(units = 128)
        self.leaky_2 = keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_3 = keras.layers.Dense(units = 256)
        self.leaky_3 = keras.layers.LeakyReLU(alpha = 0.01)
        self.output_layer = keras.layers.Dense(units=3, activation = "tanh")
        
    def call(self, input_tensor):
        ## Definition of Forward Pass
        x = self.input_layer(input_tensor)
        x = self.dense_1(x)
        x = self.leaky_1(x)
        x = self.dense_2(x)
        x = self.leaky_2(x)
        x = self.dense_3(x)
        x = self.leaky_3(x)
        return  self.output_layer(x)
    
    def generate_noise(self,batch_size, random_noise_size):
        return np.random.uniform(-1,1, size = (batch_size, random_noise_size, 3))


In [137]:
class Discriminator(keras.Model):
    def __init__(self):
        super().__init__(name = "discriminator")
        
        #Layers
        self.input_layer = keras.layers.Dense(units = 3)
        self.dense_1 = keras.layers.Dense(units = 128)
        self.leaky_1 =  keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_2 = keras.layers.Dense(units = 128)
        self.leaky_2 = keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_3 = keras.layers.Dense(units = 128)
        self.leaky_3 = keras.layers.LeakyReLU(alpha = 0.01)
        
        self.logits = keras.layers.Dense(units = 1)  # This neuron tells us if the input is fake or real
    def call(self, input_tensor):
          ## Definition of Forward Pass
        x = self.input_layer(input_tensor)
        x = self.dense_1(x)
        x = self.leaky_1(x)
        x = self.leaky_2(x)
        x = self.leaky_3(x)
        x = self.leaky_3(x)
        x = self.logits(x)
        return x

In [138]:
discriminator  = Discriminator()
generator = Generator()

### Objective

In [139]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)


In [140]:
def generator_objective(dx_of_gx):
    # Labels are true here because generator thinks he produces real images. 
    return cross_entropy(tf.ones_like(dx_of_gx), dx_of_gx)

In [141]:
def discriminator_objective(real_output, fake_output, smoothing_factor=0.9):
    """
    label = 0: fake
    label = 1: real
    """
    real_loss = cross_entropy(tf.ones_like(real_output * smoothing_factor, real_output))
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    return real_loss + fake_loss

In [142]:
generator_optimizer = keras.optimizers.RMSprop()
discriminator_optimizer = keras.optimizers.RMSprop()

In [143]:
@tf.function()
def training_step(generator: Generator, discriminator: Discriminator, images:np.ndarray , k:int=1, batch_size=32):
    for _ in range(k):
         with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            noise = generator.generate_noise(batch_size, 64)
            g_z = generator(noise)
            print(type(g_z))
            print("G_Z: ", g_z.shape)
            images = tf.cast(images, tf.float32)
            d_x_true = discriminator(images) # Trainable?
            print("IMAGES: ", images.shape)
            
            d_x_fake = discriminator(g_z) # dx_of_gx
            print(d_x_fake.shape)
            print(type(d_x_fake))
            discriminator_loss = discriminator_objective(d_x_true, d_x_fake)
            # Adjusting Gradient of Discriminator
            disc_grad = disc_tape.gradient(discriminator_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(disc_grad, discriminator.trainable_variables)) # Takes a list of gradient and variables pairs
            
              
            generator_loss = generator_objective(d_x_fake)
            # Adjusting Gradient of Generator
            gen_grad = gen_tape.gradient(generator_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gen_grad, generator.trainable_variables))

In [144]:
def training(dataset, epoches):
    for epoch in range(epoches):
        for batch in dataset: 
            print(type(batch))
            print(type(epoches))
            print(type(BATCH_SIZE))
            print(type(generator))
            print(type(discriminator))
            training_step(generator, discriminator, batch, k=1, batch_size=BATCH_SIZE)
            
        ## After ith epoch plot image 
        if (epoch % 50) == 0: 
            fake_image = tf.reshape(generator(seed), shape=(28,28))
            print("{}/{} epoches".format(epoch, epoches))
            #plt.imshow(fake_image, cmap = "gray")
            plt.imsave("{}/{}.png".format(OUTPUT_DIR,epoch),fake_image, cmap = "gray")

In [145]:
%%time
training(X_train, EPOCHS)

<class 'numpy.ndarray'>
<class 'int'>
<class 'int'>
<class '__main__.Generator'>
<class '__main__.Discriminator'>

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

<class 'tensorflow.python.framework.ops.Tensor'>
G_Z:  (256, 64, 3)
IMAGES:  (64, 64, 3)
(256, 64, 1)
<class 'tensorflow.python.framework.ops.Tensor'>


TypeError: in converted code:

    <ipython-input-143-00af8eb356cd>:16 training_step  *
        discriminator_loss = discriminator_objective(d_x_true, d_x_fake)
    <ipython-input-141-be87a2d736ce>:6 discriminator_objective  *
        real_loss = cross_entropy(tf.ones_like(real_output * smoothing_factor, real_output))
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/util/dispatch.py:180 wrapper
        
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:2613 ones_like_v2
        
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:2623 ones_like_impl
        
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:2652 ones
        
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:720 as_dtype
        
    /Users/sofiapaganin/anaconda3/lib/python3.7/s

In [146]:
tf.__version__

'2.1.0'

In [147]:
keras.__version__

'2.2.4-tf'

### Sample Outputs

In [148]:
fake_image = generator(np.random.uniform(-1,1, size = (1, 100)))
plt.imshow(tf.reshape(fake_image, shape = (28,28)), cmap="gray")

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,100], In[1]: [3,100] [Op:MatMul]